What I need in this file. Only for FreeRecall
Confidence interval over how many errors there where made.
Histogram that compares normal, speed, memory, and sees how many times the first number was wrong.
Histogram that compares normal, speed, memory, and sees how many times the last number was wrong.
Histogram that compares normal, speed, memory, errors made.
A graph over errors made in speed

Files needed
combined_memorypattern_data.csv
combined_normal_data.csv
combined_speed_data.csv